In [1]:
# Use Google Colab
use_colab = True

# Is this notebook running on Colab?
# If so, then google.colab package (github.com/googlecolab/colabtools)
# should be available in this environment

# Previous version used importlib, but we could do the same thing with
# just attempting to import google.colab
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts'
    
    # Install packages specified in requirements
    !pip install -r requirements.txt

    %cd 'micro_f1_calc'
    
    # List the directory contents
    !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts
/content/drive/My Drive/cs696ds_lexalytics/Ronald Gypsum Prompts/micro_f1_calc
bert_prompt_logit_softmax_micro_f1_calc.ipynb
collect_results_snippet.ipynb
papermill_launcher_prompt_logit_softmax.ipynb


In [2]:
import os
import itertools

import papermill
import tqdm

In [6]:
# experiment id prefix
experiment_id_prefix = 'bert_prompt_logit_softmax_micro_f1_calc'

# Random seed
random_seeds = [696, 685, 683, 682, 589]

# path to pretrained MLM model folder or the string "bert-base-uncased"
lm_model_paths = {
    #'bert_yelp_restaurants': '../trained_models/lm_further_pretraining_bert_yelp_restaurants_bseoh_2021-03-22--15_03_31',
    'bert_amazon_electronics': '../progress/lm_further_pretraining_bert_amazon_electronics_bseoh_2021-03-06--18_59_53/results/checkpoint-1180388',
    #'bert-base-uncased': 'bert-base-uncased'
}

# Prompts to be added to the end of each review text
# Note: pseudo-labels for each prompt should be given in the order of (positive), (negative), (neutral)
sentiment_prompts = {
    'i_felt': {"prompt": "I felt the {aspect} was [MASK].", "labels": ["good", "bad", "ok"]},
    'made_me_feel': {"prompt": "The {aspect} made me feel [MASK].", "labels": ["good", "bad", "indifferent"]},
    'the_aspect_is': {"prompt": "The {aspect} is [MASK].", "labels": ["good", "bad", "ok"]}
}

run_single_prompt = True
run_multiple_prompts = False

prompts_merge_behavior = 'sum_logits'
prompts_perturb = False

# Training settings
training_domain = 'laptops' # 'laptops', 'restaurants', 'joint'

# Few-shot dataset size
training_dataset_few_shot_size = 16

if training_dataset_few_shot_size == -1:
    experiment_id_prefix_override = 'bert_prompt_logit_softmax_revised_atsc'
else:
    experiment_id_prefix_override = 'bert_' + str(training_dataset_few_shot_size) + '_shot_' + 'prompt_logit_softmax_atsc'

# Test settings
testing_batch_size = 32
testing_domain = 'laptops'

if testing_domain != training_domain:
    cross_domain = True
else:
    cross_domain = False

# Results directory path
results_path = 'results_' + experiment_id_prefix_override + '_' + testing_domain
os.makedirs(results_path, exist_ok=True)

In [7]:
# Run single prompt experiments first
if run_single_prompt:
    
    print("Running single prompts...")
    print()

    for seed, lm_model_name, prompt_key in tqdm.tqdm(itertools.product(random_seeds, lm_model_paths.keys(), sentiment_prompts.keys())):
        
        # We will use the following string ID to identify this particular (training) experiments
        # in directory paths and other settings
        experiment_id = experiment_id_prefix_override + '_'
        experiment_id = experiment_id + testing_domain + '_'
        
        if cross_domain:
            experiment_id = experiment_id + 'cross_domain_'

        experiment_id = experiment_id + lm_model_name + '_'
        experiment_id = experiment_id + 'single_prompt' + '_'
        experiment_id = experiment_id + prompt_key + '_'
        experiment_id = experiment_id + str(seed)
        
        print("Running experiment", experiment_id)
        print()
        
        parameters_to_inject = {
            'experiment_id': experiment_id,
            'random_seed': seed,
            'lm_model_path': lm_model_paths[lm_model_name],
            #'training_domain': training_domain,
            'sentiment_prompts': [sentiment_prompts[prompt_key]],
            #'training_dataset_few_shot_size': training_dataset_few_shot_size,
            'testing_batch_size': testing_batch_size,
            'testing_domain': testing_domain,
            'prompts_merge_behavior': prompts_merge_behavior,
            'prompts_perturb': prompts_perturb
        }

        papermill.execute_notebook(
            os.path.join(experiment_id_prefix + '.ipynb'),
            os.path.join(results_path, experiment_id + '.ipynb'),
            parameters=parameters_to_inject,
            log_output=True,
            progress_bar=True,
            autosave_cell_every=300,
            cwd=os.getcwd(),
        )


0it [00:00, ?it/s]

Running single prompts...

Running experiment bert_16_shot_prompt_logit_softmax_atsc_laptops_bert_amazon_electronics_single_prompt_i_felt_696



Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-3ab1dc8a0753d3b8/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-956f09c80828d7d4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)


1it [00:30, 30.62s/it]


Running experiment bert_16_shot_prompt_logit_softmax_atsc_laptops_bert_amazon_electronics_single_prompt_made_me_feel_696



Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-3ab1dc8a0753d3b8/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-956f09c80828d7d4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)


2it [01:01, 30.65s/it]

...

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-3ab1dc8a0753d3b8/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-956f09c80828d7d4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)


13it [06:41, 30.91s/it]


Running experiment bert_16_shot_prompt_logit_softmax_atsc_laptops_bert_amazon_electronics_single_prompt_made_me_feel_589



Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-3ab1dc8a0753d3b8/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-956f09c80828d7d4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


14it [07:14, 31.60s/it]


Running experiment bert_16_shot_prompt_logit_softmax_atsc_laptops_bert_amazon_electronics_single_prompt_the_aspect_is_589



Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-3ab1dc8a0753d3b8/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

Using custom data configuration default

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/default-956f09c80828d7d4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)


15it [07:46, 31.09s/it]

In [ ]:
# Run multiple prompts
if run_multiple_prompts:
    print("Running multiple prompts...")
    print()

    for seed, lm_model_name in tqdm.tqdm(itertools.product(random_seeds, lm_model_paths.keys())):
        
        # We will use the following string ID to identify this particular (training) experiments
        # in directory paths and other settings
        experiment_id = experiment_id_prefix_override + '_'
        experiment_id = experiment_id + testing_domain + '_'
        
        if cross_domain:
            experiment_id = experiment_id + 'cross_domain_'

        experiment_id = experiment_id + lm_model_name + '_'
        experiment_id = experiment_id + 'multiple_prompts' + '_'
        experiment_id = experiment_id + str(seed)
        
        print("Running experiment ", experiment_id)
        print()
        
        parameters_to_inject = {
            'experiment_id': experiment_id,
            'random_seed': seed,
            'lm_model_path': lm_model_paths[lm_model_name],
            'sentiment_prompts': [sentiment_prompts[prompt_key] for prompt_key in sentiment_prompts.keys()],
            #'training_domain': training_domain,
            #'training_dataset_few_shot_size': training_dataset_few_shot_size,
            'testing_batch_size': testing_batch_size,
            'testing_domain': testing_domain,
            'prompts_merge_behavior': prompts_merge_behavior,
            'prompts_perturb': prompts_perturb
        }

        papermill.execute_notebook(
            os.path.join(experiment_id_prefix + '.ipynb'),
            os.path.join(results_path, experiment_id + '.ipynb'),
            parameters=parameters_to_inject,
            log_output=True,
            progress_bar=True,
            autosave_cell_every=300,
            cwd=os.getcwd(),
        )

0it [00:00, ?it/s]

Running multiple prompts...

Running experiment  bert_prompt_logit_softmax_revised_atsc_restaurants_bert_yelp_restaurants_multiple_prompts_696



Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

1it [01:29, 89.76s/it]


Running experiment  bert_prompt_logit_softmax_revised_atsc_restaurants_bert-base-uncased_multiple_prompts_696



Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

2it [03:07, 92.14s/it]

...

Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

8it [13:12, 99.90s/it]


Running experiment  bert_prompt_logit_softmax_revised_atsc_restaurants_bert_yelp_restaurants_multiple_prompts_589



Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

9it [14:51, 99.78s/it]


Running experiment  bert_prompt_logit_softmax_revised_atsc_restaurants_bert-base-uncased_multiple_prompts_589



Reusing dataset sem_eval2014_task4_dataset (../dataset_cache/sem_eval2014_task4_dataset/SemEval2014Task4Dataset - Subtask 4/0.0.1/537edd3b5fdbdb1f3190419cf0a53a4fab3537bc666f17c8c75fa8d0b554e529)

10it [16:33, 99.32s/it] 